<a href="https://colab.research.google.com/github/Akshat-02/Video-to-text-summary/blob/main/Akshat_CCTV_Insight_Generator_Project_FInal_Year.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Installing Huggingface Transformers

In [ ]:
!pip install git+https://github.com/huggingface/transformers.git opencv-python

  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-1frlyf9v
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-1frlyf9v
  Resolved https://github.com/huggingface/transformers.git to commit 481a95781404e48b1c80940be17e8279dec82fe8
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 6.1 MB/s eta 0:00:00
  Created wheel for transformers: filename=transformers-4.42.0.dev0-py3-none-any.whl size=9096013 sha256=d32945ddf9a41ecd62fb51615527c5e274226b821017907b19fbd74f0cefe934
  Stored in directory: /tmp/pip-ephem-wheel-cache-za_c6ezy/wheels/e7/9c/5b/e1a9c8007c343041e61cc484433d512ea9274272e3fcbe7c16
Successfully built transformers
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-

#Getting all the required LLM models

In [ ]:
#Importing PyTorch
import torch

#For Bart
from transformers import pipeline

summarizer = pipeline("summarization", model="facebook/bart-large-cnn")


#For Blip2
from transformers import AutoProcessor, Blip2ForConditionalGeneration

processor = AutoProcessor.from_pretrained("Salesforce/blip2-opt-2.7b")
model_blip2 = Blip2ForConditionalGeneration.from_pretrained("Salesforce/blip2-opt-2.7b", torch_dtype=torch.float16)

#Device agnostic
device = "cuda" if torch.cuda.is_available() else "cpu"
model_blip2.to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/127k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/10.0G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/5.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Blip2ForConditionalGeneration(
  (vision_model): Blip2VisionModel(
    (embeddings): Blip2VisionEmbeddings(
      (patch_embedding): Conv2d(3, 1408, kernel_size=(14, 14), stride=(14, 14))
    )
    (encoder): Blip2Encoder(
      (layers): ModuleList(
        (0-38): 39 x Blip2EncoderLayer(
          (self_attn): Blip2Attention(
            (dropout): Dropout(p=0.0, inplace=False)
            (qkv): Linear(in_features=1408, out_features=4224, bias=True)
            (projection): Linear(in_features=1408, out_features=1408, bias=True)
          )
          (layer_norm1): LayerNorm((1408,), eps=1e-06, elementwise_affine=True)
          (mlp): Blip2MLP(
            (activation_fn): GELUActivation()
            (fc1): Linear(in_features=1408, out_features=6144, bias=True)
            (fc2): Linear(in_features=6144, out_features=1408, bias=True)
          )
          (layer_norm2): LayerNorm((1408,), eps=1e-06, elementwise_affine=True)
        )
      )
    )
    (post_layernorm): LayerNorm((

#Video Feed Input and Text Generation

In [ ]:
#Importing necessary libraries
import cv2
from PIL import Image

#Capturing video from source
file_path = "/content/test_vid_2.mp4"
cap = cv2.VideoCapture(file_path)

if not cap.isOpened():
  print("Error loading Video")
  exit()

counter = 0
vid_desc = ''
# Read until the video is completed
while cap.isOpened():

  #Capture frame-by-frame
  ret, frame = cap.read()
  if ret:
    if counter % 15 == 0 and frame is not None:
      rgb_image = Image.fromarray(frame.astype('uint8'), 'RGB')

      inputs = processor(rgb_image, return_tensors="pt").to(device, torch.float16)

      generated_ids = model_blip2.generate(**inputs, max_new_tokens=50)
      generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0].strip()

      print(generated_text.capitalize())
      vid_desc += generated_text.capitalize() + '.\n'

    counter += 1

  else:
    break
# Release the VideoCapture object
cap.release()

# Close all windows
cv2.destroyAllWindows()

Both `max_new_tokens` (=50) and `max_length`(=51) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=50) and `max_length`(=51) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


A mountain of garbage is being worked on by a bulldozer


Both `max_new_tokens` (=50) and `max_length`(=51) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


A group of blue construction equipment on top of a mountain


Both `max_new_tokens` (=50) and `max_length`(=51) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


...

A man is laying bricks on a brick patio


Both `max_new_tokens` (=50) and `max_length`(=51) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


A group of men working on a brick patio


Both `max_new_tokens` (=50) and `max_length`(=51) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


A man is laying bricks on a brick patio
A man is laying bricks on a brick patio


In [ ]:
#Actual Generated text for each frame:
print(vid_desc)

A mountain of garbage is being worked on by a bulldozer.
A group of blue construction equipment on top of a mountain.
A pile of garbage with blue construction equipment on top.
A group of blue construction equipment is working on a pile of garbage.
A pile of garbage with blue construction equipment on top.
A pile of garbage with blue construction equipment on top.
A garbage truck is parked on the side of the road.
A video game showing a garbage truck driving down a street.
A garbage truck is parked on the side of the road.
A man is standing next to a garbage truck.
A garbage truck is being loaded onto a truck.
Garbage truck with a man standing next to it.
A garbage truck is being loaded into a container.
A garbage truck is being loaded into a truck.
A garbage truck is dumping trash into a large container.
A man in a yellow jacket and hard hat standing in front of a control panel.
A man in a hard hat and safety glasses standing in front of a control panel.
A large blue machine with a bu

#Text Summarization

##Handling duplicates

In [ ]:
#Managing duplicate generation
vid_desc_list = list(set(vid_desc.split("\n")))
vid_desc_final = "\n".join(vid_desc_list)

print(f"Final Video Description:\n\n{vid_desc_final}")

Final Video Description:


A large pile of garbage in a factory.
A computer generated image of a blue cloud.
A city skyline with the words electric city.
A group of blue construction equipment is working on a pile of garbage.
A video game showing a garbage truck driving down a street.
A blue sky with clouds and the word 'these' written on it.
A blue fire is seen in the middle of a dark room.
A large blue machine with a large claw on it.
A city skyline at night with the words generated.
A video of a machine that is being used to make a pipe.
The city skyline is lit up at night with the words becomes.
A pile of black sand with the word theyy on it.
A man in a yellow jacket and hard hat standing in front of a control panel.
Garbage truck with a man standing next to it.
A blue light is shining in the dark with the word trash.
A man is standing in front of a sign that says new sand.
The batman v superman trailer shows a scene of a man in a room with a lot of garbage.
A blue machine with a b

##Summary Generation

In [ ]:
#Getting Final summary

ARTICLE = vid_desc_final
summary_text = summarizer(ARTICLE, max_length=300, min_length=250, do_sample=False)[0]['summary_text']

print(summary_text)

The batman v superman trailer shows a scene of a man in a room with a lot of garbage. A city skyline is lit up at night with the words electric city on it. A man is standing in front of a sign that says new sand. A blue fire is burning in a dark room. A pile of dirt with the word island in blue. A group of men working on a brick patio. A large pile of garbage in a factory. A machine that is being used to make a pipe. A garbage truck parked on the side of the road. A mountain of garbage being worked on by a bulldozer. A big blue machine with a large claw on it and a blue and yellow water with a sign saying 'out' in the background. The word 'toxic' is written on a computer screen with a blue cloud in the Background. The words 'electric city' and 'heat' are written on the city skyline at night. The city skyline has a blue light shining in the middle of the city. The name of a city is written in blue on a blue sky with clouds and the word 'these' written on it, and the city is called 'Elec

## Creating a Summary text file

In [ ]:
#Writing to output file

with open('summary.txt', 'w') as f:
    f.writelines(line + "\n" for line in summary_text.split(". "))